In [1]:
import joblib

MODEL_PATH = "../backend/models/lightgbm_production_artifact_20250415_081218.pkl"

# Chargement
artifacts = joblib.load(MODEL_PATH)

# Inspection des clés principales
print("🔍 Clés disponibles dans artifacts :", artifacts.keys())

# Si metadata est bien là, on regarde ce qu'il contient
metadata = artifacts.get("metadata", {})
print("🔍 Clés disponibles dans metadata :", metadata.keys())

# On vérifie si les features sont bien définies
features = metadata.get("features", None)
if features is None:
    print("❌ 'features' est manquant dans metadata.")
else:
    print(f"✅ Nombre de features : {len(features)}")
    print("Extrait des features :", features[:5])  # Affiche les 5 premières



🔍 Clés disponibles dans artifacts : dict_keys(['scaler', 'model', 'metadata'])
🔍 Clés disponibles dans metadata : dict_keys(['optimal_threshold', 'training_date', 'features', 'dtype_example', 'training_stats', 'performance_metrics', 'training_strategy'])
✅ Nombre de features : 85
Extrait des features : ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'AMT_CREDIT', 'ANNUITY_INCOME_PERC']


In [6]:
import pickle

# Charge le fichier artifact contenant les informations
artifact_path = "../backend/models/lightgbm_production_artifact_20250415_081218.pkl"

# Charger les artifacts
with open(artifact_path, 'rb') as f:
    artifacts = pickle.load(f)

# Afficher le type de l'objet chargé pour mieux comprendre sa structure
print("🔍 Type des artifacts :", type(artifacts))

# Afficher un extrait des données pour comprendre la structure
print("🔍 Extrait des artifacts :", artifacts[:5])  # Affiche les 5 premiers éléments si c'est un tableau
print("longueur du tableau" ,len(artifacts))


🔍 Type des artifacts : <class 'numpy.ndarray'>
🔍 Extrait des artifacts : ['EXT_SOURCE_1' 'EXT_SOURCE_2' 'EXT_SOURCE_3' 'AMT_CREDIT'
 'ANNUITY_INCOME_PERC']
longueur du tableau 85


In [9]:
import pandas as pd

# Suppose que client_data est un DataFrame que tu veux prédire
TEST_SAMPLE_FOR_API = "../backend/data/test_1000_sample_for_api.csv"
TEST_SAMPLE_WITH_TARGET = "../backend/data/test_1000_sample_with_target.csv"

df_test = pd.read_csv(TEST_SAMPLE_WITH_TARGET)
df_api = pd.read_csv(TEST_SAMPLE_FOR_API)

client_id = df_test["SK_ID_CURR"]
client_data = df_api[df_api["SK_ID_CURR"] == client_id].drop(columns=["SK_ID_CURR"])

# Comparer les features attendues avec les données d'entrée
expected_features = artifacts

# Trouver les colonnes manquantes dans les données d'entrée
missing_cols = set(expected_features) - set(client_data.columns)

# Afficher les résultats
if missing_cols:
    print("⚠️ Colonnes manquantes dans les données d'entrée :", missing_cols)
else:
    print("✅ Toutes les colonnes nécessaires sont présentes dans les données.")


✅ Toutes les colonnes nécessaires sont présentes dans les données.


In [19]:
import joblib

# Charger les artefacts sauvegardés
model_path = "../backend/models/lightgbm_production_artifact_20250415_081218.pkl"
artifacts = joblib.load(model_path)

# Extraire le modèle et le scaler
model = artifacts['model']
scaler = artifacts['scaler']

# Si nécessaire, obtenir les métadonnées
metadata = artifacts['metadata']
optimal_threshold = metadata['optimal_threshold']
features = metadata['features']

# Vérifier les données d'entrée (client_data)
client_data = client_data[features]  

# Appliquer le scaler sur les données du client
client_data_scaled = scaler.transform(client_data)

# Appliquer le scaler en conservant les noms de colonnes
client_data_scaled = pd.DataFrame(
    scaler.transform(client_data),
    columns=client_data.columns,
    index=client_data.index
)

# Faire une prédiction sur les données du client
pred_proba = model.predict_proba(client_data_scaled)[:, 1]  # Probabilité de défaut

# Appliquer le seuil optimal pour la prédiction finale
pred = (pred_proba >= optimal_threshold).astype(int)

# Affichage des résultats
print(f"Probabilité de défaut : {pred_proba[0]:.2%}")
print(f"Décision : {'🛑 Refusé' if pred[0] == 1 else '✅ Accepté'}")


Probabilité de défaut : 3.91%
Décision : ✅ Accepté


In [16]:
import pickle
import lightgbm as lgb

# Charge les artifacts depuis le fichier
artifact_path = "../backend/models/lightgbm_production_artifact_20250415_081218.pkl"

# Charger les artifacts
with open(artifact_path, 'rb') as f:
    artifacts = pickle.load(f)

# Vérifier la structure des artifacts
print("🔍 Type des artifacts :", type(artifacts))
print("🔍 Forme des artifacts :", artifacts.shape)  # Vérifie si c'est un tableau NumPy et sa taille

# Afficher un extrait pour mieux comprendre la structure
print("🔍 Extrait des artifacts :", artifacts[:5])  # Affiche les 5 premiers éléments si c'est un tableau


🔍 Type des artifacts : <class 'numpy.ndarray'>
🔍 Forme des artifacts : (85,)
🔍 Extrait des artifacts : ['EXT_SOURCE_1' 'EXT_SOURCE_2' 'EXT_SOURCE_3' 'AMT_CREDIT'
 'ANNUITY_INCOME_PERC']


In [20]:
import requests
import pandas as pd
import random
import json

# URL de ton API (modifie selon ton environnement)
API_URL = "http://localhost:8000/predict"

# Chargement des données test
df_test = pd.read_csv("../backend/data/test_1000_sample_for_api.csv")

# Sélection aléatoire d'un client
random_client = df_test.sample(n=1)  # Sélectionne un seul client au hasard
client_data = random_client.drop(columns=["SK_ID_CURR"]).to_dict(orient="records")[0]  # Extrait les données du client sous forme de dictionnaire

# Affichage des données du client sélectionné
print("Données du client sélectionné :")
print(client_data)

# Construction du corps de la requête
data = {
    "data": client_data  # Les données du client sélectionné
}

# Envoi de la requête à l'API
response = requests.post(API_URL, json=data)

# Affichage de la réponse de l'API
if response.status_code == 200:
    result = response.json()
    print("Réponse de l'API :")
    print(json.dumps(result, indent=4))
else:
    print(f"Erreur {response.status_code} : {response.text}")



Données du client sélectionné :
{'EXT_SOURCE_1': 0.5074440332442849, 'EXT_SOURCE_2': 0.7109608784019691, 'EXT_SOURCE_3': 0.7267112092725122, 'AMT_CREDIT': 1113840.0, 'ANNUITY_INCOME_PERC': 0.4222333333333333, 'BURO_DAYS_CREDIT_ENDDATE_MEAN': -65.0, 'BURO_DAYS_CREDIT_UPDATE_MEAN': -72.0, 'DAYS_BIRTH': -19274.0, 'DAYS_ID_PUBLISH': -2805.0, 'FLAG_EMP_PHONE': 1.0, 'NAME_INCOME_TYPE_PENSIONER': 0.0, 'BURO_AMT_CREDIT_SUM_MAX': 485640.0, 'BURO_CREDIT_ACTIVE_CLOSED_MEAN': 1.0, 'BURO_CREDIT_TYPE_CAR_LOAN_MEAN': 0.0, 'BURO_CREDIT_TYPE_CREDIT_CARD_MEAN': 0.0, 'BURO_CREDIT_TYPE_MICROLOAN_MEAN': 0.0, 'BURO_CREDIT_TYPE_MORTGAGE_MEAN': 0.0, 'BURO_DAYS_CREDIT_ENDDATE_MAX': -65.0, 'BURO_DAYS_CREDIT_MEAN': -1161.0, 'CODE_GENDER': 1.0, 'DAYS_EMPLOYED_PERC': 0.2647089343156584, 'DAYS_LAST_PHONE_CHANGE': -802.0, 'DAYS_REGISTRATION': -9316.0, 'DEF_60_CNT_SOCIAL_CIRCLE': 0.0, 'FLAG_DOCUMENT_3': 1.0, 'FLAG_DOCUMENT_6': 0.0, 'NAME_HOUSING_TYPE_RENTED_APARTMENT': 0.0, 'OCCUPATION_TYPE_ACCOUNTANTS': 0.0, 'ORGANI

In [21]:

import requests
import json

API_URL = "http://localhost:8000/predict"

{
  "data": {
    "EXT_SOURCE_1": 0.5074440332442849,
    "EXT_SOURCE_2": 0.7109608784019691,
    "EXT_SOURCE_3": 0.7267112092725122,
    "AMT_CREDIT": 1113840.0,
    "ANNUITY_INCOME_PERC": 0.4222333333333333,
    "BURO_DAYS_CREDIT_ENDDATE_MEAN": -65.0,
    "BURO_DAYS_CREDIT_UPDATE_MEAN": -72.0,
    "DAYS_BIRTH": -19274.0,
    "DAYS_ID_PUBLISH": -2805.0,
    "FLAG_EMP_PHONE": 1.0,
    "NAME_INCOME_TYPE_PENSIONER": 0.0,
    "BURO_AMT_CREDIT_SUM_MAX": 485640.0,
    "BURO_CREDIT_ACTIVE_CLOSED_MEAN": 1.0,
    "BURO_CREDIT_TYPE_CAR_LOAN_MEAN": 0.0,
    "BURO_CREDIT_TYPE_CREDIT_CARD_MEAN": 0.0,
    "BURO_CREDIT_TYPE_MICROLOAN_MEAN": 0.0,
    "BURO_CREDIT_TYPE_MORTGAGE_MEAN": 0.0,
    "BURO_DAYS_CREDIT_ENDDATE_MAX": -65.0,
    "BURO_DAYS_CREDIT_MEAN": -1161.0,
    "CODE_GENDER": 1.0,
    "DAYS_EMPLOYED_PERC": 0.2647089343156584,
    "DAYS_LAST_PHONE_CHANGE": -802.0,
    "DAYS_REGISTRATION": -9316.0,
    "DEF_60_CNT_SOCIAL_CIRCLE": 0.0,
    "FLAG_DOCUMENT_3": 1.0,
    "FLAG_DOCUMENT_6": 0.0,
    "NAME_HOUSING_TYPE_RENTED_APARTMENT": 0.0,
    "OCCUPATION_TYPE_ACCOUNTANTS": 0.0,
    "ORGANIZATION_TYPE_BUSINESS_ENTITY_TYPE_3": 1.0,
    "ORGANIZATION_TYPE_MILITARY": 0.0,
    "ORGANIZATION_TYPE_SELF_EMPLOYED": 0.0,
    "PAYMENT_RATE": 0.0511756625727213,
    "REGION_POPULATION_RELATIVE": 0.04622,
    "REGION_RATING_CLIENT_W_CITY": 1.0,
    "REG_CITY_NOT_LIVE_CITY": 0.0,
    "ACTIVE_DAYS_CREDIT_ENDDATE_MEAN": 716.0,
    "AMT_REQ_CREDIT_BUREAU_QRT": 0.0,
    "APPROVED_AMT_ANNUITY_MEAN": 12188.835,
    "APPROVED_DAYS_DECISION_MIN": -802.0,
    "APPROVED_HOUR_APPR_PROCESS_START_MEAN": 14.666666666666666,
    "BURO_AMT_CREDIT_MAX_OVERDUE_MEAN": 0.0,
    "BURO_AMT_CREDIT_SUM_DEBT_SUM": 0.0,
    "BURO_CREDIT_TYPE_ANOTHER_TYPE_OF_LOAN_MEAN": 0.0,
    "BURO_CREDIT_TYPE_LOAN_FOR_BUSINESS_DEVELOPMENT_MEAN": 0.0,
    "BURO_CREDIT_TYPE_LOAN_FOR_THE_PURCHASE_OF_EQUIPMENT_MEAN": 0.0,
    "BURO_DAYS_CREDIT_MAX": -1161.0,
    "BURO_DAYS_CREDIT_VAR": 386627.0666666667,
    "CLOSED_AMT_CREDIT_SUM_SUM": 485640.0,
    "ELEVATORS_AVG": 0.0,
    "FLAG_DOCUMENT_13": 0.0,
    "FLAG_DOCUMENT_15": 0.0,
    "FLAG_DOCUMENT_16": 0.0,
    "FLAG_DOCUMENT_17": 0.0,
    "FLAG_DOCUMENT_20": 0.0,
    "FLAG_DOCUMENT_21": 0.0,
    "FLAG_OWN_CAR": 0.0,
    "FLAG_WORK_PHONE": 0.0,
    "FLOORSMAX_MODE": 0.1667,
    "INCOME_CREDIT_PERC": 0.12120232708468,
    "INCOME_PER_PERSON": 67500.0,
    "INSTAL_AMT_PAYMENT_MAX": 16267.815,
    "INSTAL_AMT_PAYMENT_MEAN": 13673.899285714286,
    "INSTAL_AMT_PAYMENT_MIN": 547.2,
    "INSTAL_AMT_PAYMENT_SUM": 382869.18,
    "INSTAL_DAYS_ENTRY_PAYMENT_MAX": -36.0,
    "INSTAL_DAYS_ENTRY_PAYMENT_SUM": -10352.0,
    "INSTAL_DBD_MAX": 29.0,
    "INSTAL_DBD_MEAN": 4.928571428571429,
    "INSTAL_DBD_SUM": 138.0,
    "INSTAL_DPD_MEAN": 0.4285714285714285,
    "INSTAL_PAYMENT_DIFF_MEAN": 514.0446428571429,
    "INSTAL_PAYMENT_DIFF_SUM": 14393.25,
    "LIVE_CITY_NOT_WORK_CITY": 0.0,
    "LIVINGAREA_MODE": 0.0731,
    "NAME_EDUCATION_TYPE_ACADEMIC_DEGREE": 0.0,
    "NAME_EDUCATION_TYPE_SECONDARY__SECONDARY_SPECIAL": 1.0,
    "NAME_FAMILY_STATUS_MARRIED": 1.0,
    "NAME_HOUSING_TYPE_CO_OP_APARTMENT": 0.0,
    "NAME_HOUSING_TYPE_MUNICIPAL_APARTMENT": 0.0,
    "NAME_INCOME_TYPE_STUDENT": 0.0,
    "NAME_INCOME_TYPE_WORKING": 0.0,
    "NAME_TYPE_SUITE_OTHER_B": 0.0,
    "OCCUPATION_TYPE_DRIVERS": 0.0,
    "OCCUPATION_TYPE_LABORERS": 1.0,
    "OCCUPATION_TYPE_LOW_SKILL_LABORERS": 0.0
  }
}

response = requests.post(API_URL, json=data)
print(response.status_code)
print(response.json())


200
{'probability': 2.69, 'decision': '✅ Accepté'}
